# Implementation of One shot learning
    Opencv is used for pre-processing the images prior training
    Transfer learning is used to replicate the Openface model structure and pretrained weights are taken for this purpose
    

#  What is One shot learning?
    One short learning is where only one image per person is stored in database, which is passed through the neural network model built and an embedding vector is generated for each person in DB.
    New embedding is created for person and compared with the stored embedding through euclidean distance. If there exist similarities between two vectors, person is recognized with the respective name.

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import glob
from imutils import paths
import cv2
import os
import numpy as np
from numpy import genfromtxt
from keras.utils import CustomObjectScope
import pygame
import pandas as pd
import tensorflow as tf
import utils
import pickle
import model
from model import Model_Architecture

In [0]:
model = Model_Architecture()

In [0]:
weights = utils.weights
weights_dict = utils.load_weights()

In [0]:
for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

In [0]:
# save model weights
model.save('./weights1.h5')

In [0]:
# to load save weights
with CustomObjectScope({'tf': tf}):
    model = load_model('./weights1.h5')

# Capturing Images at instance for training
    This helps capture upto 10 frames of a person. Further, any one image could be kept for training in database

In [0]:
cam = cv2.VideoCapture(0)

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

count = 0
while(True):
    ret, img = cam.read()
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite('C://Users//Abhishek//Face_recognition//New_dataset//Sri//Srii_' + str(count) + ".jpg", img[y1:y2,x1:x2])
        cv2.imshow('image', img)
    k = cv2.waitKey(200) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 10: # Take 30 face sample and stop video
         break
    
cam.release()
cv2.destroyAllWindows()
            

# Create embeddings for images

In [0]:
def image_to_embedding(image, model):
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

# Recognizing Faces
    For recognizing new images in real-time, euclidean distance is calculated. 
    Euclidean distance finds distance of two vectors on an euclidean space.
    Comparing the similarity of captured image embedding and stored train embeddings, the character's name is predicted where the similarity is more. 
    HAAR filter in opencv is used to create the bounding box across face during the process.

In [0]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model) #changed for testing
    
    minimum_distance = 200
    name = None
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    if minimum_distance < 0.80:
        return str(name)
    else:
        return str("Unknown")

In [0]:

def create_input_image_embeddings():
    input_embeddings = {}
    path =[]
    path = list(paths.list_images('C://Users//Abhishek//Face_recognition//New_dataset'))
    for(i, imagePath) in enumerate(path):
        person_name = imagePath.split(os.path.sep)[-2]
        image_file = cv2.imread(imagePath)
        input_embeddings[person_name] = image_to_embedding(image_file, model)
        
    return input_embeddings

**Additional feature added**

Sound effects are added which could be used for security purpose.
Incase of a person's face being deteced, notification plays saying access granted and in case of unknown person being detected, warning sound plays.


In [0]:
import datetime as dt
pygame.mixer.init()
pygame.mixer.set_num_channels(8)
voice = pygame.mixer.Channel(2)
s1 = pygame.mixer.Sound("detected.wav")
s2 = pygame.mixer.Sound("warning.wav")

# Real-time recognition of faces

In [0]:

def recognize_faces_in_cam(input_embeddings):
    

    cv2.namedWindow("Face Recognition")
    vc = cv2.VideoCapture(0)
   

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h
            
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            
  
            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
                if voice.get_busy()== False:
                    if(cur_time - begin_time).seconds > 10:
                        begin_time = dt.datetime.now()
                        voice.play(s1)
            else:
                cv2.putText(img,"Unknown",(x1+5,y1-5), font, 1, (255,255,0), 1)
                if voice.get_busy()== False:
                    if(cur_time - begin_time).seconds > 3:
                        begin_time = dt.datetime.now()
                        voice.play(s2)
        
        key = cv2.waitKey(100)
        cv2.imshow("Face Recognization", img)

        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

In [0]:
input_embeddings = create_input_image_embeddings()

Input embeddings can also be saved to disk for future use

In [0]:
with open('train_faces1.dat', 'wb') as f:
    pickle.dump(input_embeddings, f)

In [0]:
recognize_faces_in_cam(input_embeddings)

To load saved input embeddings for train data

In [0]:
with open('train_faces1.dat', 'rb') as f:
    train_encodings = pickle.load(f)

# Extract list of names and the list of encodings
face_names = list(train_encodings.keys())
face_encodings = np.array(list(train_encodings.values()))